In [1]:
!pip install google-api-python-client
!pip install anthropic

from googleapiclient.discovery import build
import concurrent.futures
import ast
import os
import re
import anthropic
import requests
import json

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key
GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

client = anthropic.Anthropic()

In [14]:
def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(prompt, model="claude-3-sonnet-20240229", max_tokens=2000, temperature=0.7):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "system": "You are a world-class researcher. Analyze the given information and generate a well-structured report.",
        "messages": [{"role": "user", "content": prompt}],
    }
    try:
        response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
        response_json = response.json()
        print(response_json)
        response_text = response_json['content'][0]['text']
        print(remove_first_line(response_text.strip()))
        return remove_first_line(response_text.strip())
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")

def search_web(search_term, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    result = service.cse().list(q=search_term, cx=cse_id).execute()
    # Parse the JSON data
    parsed_data = json.loads(json.dumps(result))
    # Extract relevant text from the items
    relevant_text = []
    for item in parsed_data['items']:
        title = item['title']
        snippet = item.get('snippet', '')  # Use get() to handle missing 'snippet' key
        relevant_text.append(f"Title: {title}\nSnippet: {snippet}\n")
    # Print the relevant text
    print("\n".join(relevant_text))
    return relevant_text

def generate_subtopic_report(research_topic, subtopic):
    search_data = []
    all_queries = []
    search_cache = {}
    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 5 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
 
    initial_queries_response = generate_text(initial_queries_prompt)
    if initial_queries_response.startswith('[') and ']' in initial_queries_response:
        try:
            initial_queries = ast.literal_eval(initial_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping initial queries.")
            initial_queries = []
    else:
        print("Error: Search query format not found. Skipping initial queries.")
        initial_queries = []
   
    print(initial_queries)
    all_queries.extend(initial_queries)
    def search_and_cache(query):
        if query in search_cache:
            return search_cache[query]
        else:
            search_results = search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)
            search_cache[query] = search_results
            return search_results
    with concurrent.futures.ThreadPoolExecutor() as executor:
        search_results = list(executor.map(search_and_cache, initial_queries))
        search_data.extend(search_results)
 
    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}' of the research topic '{research_topic}':\n\n{str(search_data)}"
    report = generate_text(report_prompt, max_tokens=4000)
    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt = f"Analyze the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' and identify areas that need more detail or further information:\n\n{report}\n\n---\n\nHere are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\nGenerate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
   
    additional_queries_response = generate_text(analysis_prompt)
    if additional_queries_response.startswith('[') and ']' in additional_queries_response:
        try:
            additional_queries = ast.literal_eval(additional_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping additional queries.")
            additional_queries = []
    else:
        print("Error: Search query format not found. Skipping additional queries.")
        additional_queries = []
   
    all_queries.extend(additional_queries)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        additional_search_results = list(executor.map(search_and_cache, additional_queries))
        search_data.extend(additional_search_results)
    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt = f"Update the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data:\n\n{str(additional_search_results)}\n\n---\n\nGenerate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
    report = generate_text(update_prompt, max_tokens=4000)
    print(f"Final report generated for subtopic: {subtopic}!")
    return report

def generate_comprehensive_report(research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt = f"Generate a comprehensive report on the research topic '{research_topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the research topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report = generate_text(comprehensive_report_prompt, model="claude-3-sonnet-20240229", max_tokens=4000)
    print("Comprehensive report generated!")
    return comprehensive_report

# User input
research_topic = input("Enter the research topic: ")

# Generate subtopic checklist
subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist in a Python-parseable list. Return nothing but the list. Do so in one line. Maximum five sub-topics. Start your response with [\""
subtopic_checklist = ast.literal_eval('[' + generate_text(subtopic_checklist_prompt).split('[')[1])
print(f"Subtopic Checklist: {subtopic_checklist}")

# Generate reports for each subtopic
subtopic_reports = []
for subtopic in subtopic_checklist:
    subtopic_report = generate_subtopic_report(research_topic, subtopic)
    subtopic_reports.append(subtopic_report)

# Combine subtopic reports into a comprehensive report
comprehensive_report = generate_comprehensive_report(research_topic, "\n\n".join(subtopic_reports))

# Save the comprehensive report to a file
with open("comprehensive_report.txt", "w") as file:
    file.write(comprehensive_report)
print("Comprehensive report saved as 'comprehensive_report.txt'.")

{'id': 'msg_014SjbwgkaQMv3HWuWMhZE3F', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': '["Causes and risk factors", "Symptoms and diagnosis", "Treatment options (medication, therapy, lifestyle changes)", "Impact on daily life and relationships", "Coping strategies and support"]'}], 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 83, 'output_tokens': 43}}
["Causes and risk factors", "Symptoms and diagnosis", "Treatment options (medication, therapy, lifestyle changes)", "Impact on daily life and relationships", "Coping strategies and support"]
Subtopic Checklist: ['Causes and risk factors', 'Symptoms and diagnosis', 'Treatment options (medication, therapy, lifestyle changes)', 'Impact on daily life and relationships', 'Coping strategies and support']
Generating initial search queries for subtopic: Causes and risk factors...
{'id': 'msg_01NMqxctPbQePBQokEr9pVjx', 'type': 'message', 'role': 'assis